<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/Experiment_ask/fine_tuning/pubmed/askApatient_folder0_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1000      0 --:--:-- --:--:-- --:--:--   996
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [19]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
testing.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.459772,4.784644,0.234020,11:56


In [26]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.663173,4.479177,0.261633,12:07


In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.361162,4.442892,0.266032,12:06


In [28]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.318186,4.418154,0.267424,11:57


In [0]:
learn.save_encoder('ft_fold0_first')

In [31]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.727346,3.651783,0.416568,01:07


In [32]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.891459,3.108295,0.479290,01:31


In [33]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.406085,2.869444,0.534911,03:43


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.249048,2.650746,0.553846,03:51
1,1.976568,2.558637,0.576331,03:43


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.903059,2.417414,0.588166,03:53
1,1.727931,2.353555,0.598817,03:33


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.599975,2.288246,0.602367,03:57
1,1.499178,2.259424,0.609467,03:55


In [37]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.452826,2.224348,0.620118,04:03
1,1.379588,2.174894,0.621302,03:38


In [38]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.415385,2.176491,0.624852,03:43
1,1.352426,2.122980,0.623669,03:53


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.253443,2.162819,0.623669,03:46
1,1.317187,2.116740,0.631953,03:35
2,1.196872,2.057445,0.639053,03:42
3,1.087492,2.052699,0.643787,03:26


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.097114,2.102409,0.626036,03:57
1,1.174531,2.086621,0.631953,03:47
2,1.100475,2.052655,0.628402,03:40
3,1.074266,2.054681,0.634320,03:55


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.007972,2.103742,0.631953,03:46
1,1.080022,2.102516,0.647337,03:46
2,1.001995,2.078335,0.647337,03:44
3,1.001065,2.093042,0.648521,03:35


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.987422,2.080656,0.646154,03:38
1,0.998191,2.067438,0.662722,03:34
2,0.955534,2.052199,0.641420,03:50
3,0.906259,2.061970,0.654438,03:51


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.928819,2.097299,0.657988,03:34
1,0.997080,2.095675,0.642604,03:45
2,0.919547,2.135823,0.657988,03:56
3,0.921584,2.139733,0.656805,03:47


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.973773,2.200484,0.647337,03:38
1,0.965201,2.182304,0.652071,03:24
2,0.919725,2.155360,0.654438,03:31
3,0.935529,2.143645,0.657988,03:35


In [46]:
learn.data = data_clas
learn.get_preds(ds_type = DatasetType.Test, ordered=True)

[tensor([[1.9527e-05, 5.0968e-05, 9.3448e-05,  ..., 8.4975e-06, 1.6409e-05,
          6.4133e-03],
         [2.2044e-05, 8.9543e-06, 9.1259e-02,  ..., 1.5165e-05, 2.7465e-05,
          2.2650e-05],
         [1.6342e-07, 2.2580e-06, 1.6584e-06,  ..., 2.7186e-07, 1.9774e-07,
          2.5492e-06],
         ...,
         [1.1778e-08, 2.5763e-07, 5.1063e-07,  ..., 4.2282e-08, 2.3318e-06,
          2.1564e-03],
         [1.0929e-04, 3.0482e-04, 6.3515e-07,  ..., 2.1226e-05, 4.7055e-06,
          1.8270e-04],
         [1.1208e-09, 1.4292e-09, 1.5043e-09,  ..., 3.8353e-10, 6.2978e-09,
          9.9985e-01]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [54]:
learn.show_results(ds_type = DatasetType.Train)

text,target,prediction
xxbos when i went to xxunk down instead of xxunk xxunk i would almost fall down in the xxunk no control no strength xxunk upon xxunk up i had to hold on to xxunk to get up,100,100
xxbos top outer thigh muscle xxunk to xxunk very xxunk xxunk then get xxunk and lastly it xxunk to xxunk very xxunk xxunk xxmaj like the xxunk hot patch xxunk,771,771
xxbos not able to xxunk xxunk like whether i had done xxunk or where i had placed xxunk,809,809
xxbos xxunk and the xxunk of xxunk xxunk xxunk like i was walking on hot xxunk,127,127
xxbos xxmaj xxunk xxunk xxunk like i have stone xxunk just in xxunk of the xxunk,182,182
